# Decision Tree Method for ICD Classification

In [28]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

In [29]:
csvFileName = "../Data/2022 Initial ICD-10-CM Mappings.csv"
DESCRIPTION = "Description"
ICDCODE = "Diagnosis\nCode"

df = pd.read_csv(csvFileName, delimiter=',', skiprows=3, skipfooter=7)
df.head()

/var/folders/gk/3jvnj9f55ml5l1y3kg3b_c300000gn/T/ipykernel_75858/386889054.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(csvFileName, delimiter=',', skiprows=3, skipfooter=7)


,Diagnosis\nCode,Description,CMS-HCC\nModel\nCategory\nV21,CMS-HCC\nModel\nCategory\nV22,CMS-HCC\nModel\nCategory\nV24,RxHCC\nModel\nCategory\nV05,CMS-HCC\nESRD Model\nfor 2021\nPayment\nYear,CMS-HCC\nModel\nCategory\nV22 for 2021\nPayment Year,CMS-HCC\nModel\nCategory\nV24 for 2021\nPayment Year,RxHCC\nModel\nfor 2021\nPayment\nYear
0,A0103,Typhoid pneumonia,115.0,115.0,115.0,NaN,Yes,Yes,Yes,No
1,A0104,Typhoid arthritis,39.0,39.0,39.0,NaN,Yes,Yes,Yes,No
2,A0105,Typhoid osteomyelitis,39.0,39.0,39.0,NaN,Yes,Yes,Yes,No
3,A021,Salmonella sepsis,2.0,2.0,2.0,NaN,Yes,Yes,Yes,No
4,A0222,Salmonella pneumonia,115.0,115.0,115.0,NaN,Yes,Yes,Yes,No


In [30]:

# get descriptions
descs = df[DESCRIPTION]
ICDs = df[ICDCODE]
ICDNum = len(descs)

## Description to wordvector

In [31]:
wordSet = set()

def cleanText(text):
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('.', '')
    text = text.replace(',', '')
    return text.lower()

def add2WordSet(text):
    text = cleanText(text)
    global wordSet
    wordSet = wordSet.union(set(text.split()))

_ = descs.apply(add2WordSet)


In [32]:
word2idx = {}
idx2word = []

for wordid, word in enumerate(wordSet):
    word2idx[word] = wordid
    idx2word.append(word)


## Diabete 
ICDs start with E08-E13

In [56]:
tgtdf = None
for prefix in ["E08", "E09", "E10", "E11"]:
    if tgtdf is None:
        tgtdf = df[ICDs.str.startswith(prefix)]
    else:
        tgtdf = tgtdf.append(df[ICDs.str.startswith(prefix)])
# tgtdf = df

In [57]:
# Turn Diabete descriptions to word vectors
X = np.zeros((len(tgtdf), len(wordSet)))
Y = np.zeros(len(tgtdf))
counter = 0

def turnDesc2wordvec(text):
    global counter
    text = cleanText(text)
    for word in text.split():
        if word in word2idx:
            idx = word2idx[word]
            X[counter][idx] = 1
    Y[counter] = counter
    counter += 1

_ = tgtdf[DESCRIPTION].apply(turnDesc2wordvec)

In [58]:
clf = DecisionTreeClassifier().fit(X, Y)

## Test

In [59]:
testDescs = [
    "Diabetes mellitus due to underlying condition with hyperosmolarity with coma", # E0801
    "Diabetes mellitus due to underlying condition with unspecified diabetic retinopathy without macular edema", # E08319
    "Drug or chemical induced diabetes mellitus with mild nonproliferative diabetic retinopathy with macular edema, right eye", # E093211
    "Diabetes mellitus due to underlying condition with mild nonproliferative diabetic retinopathy with macular edema, right eye",
    "Mike is suffering from Diabetes due to a long time condition of hyperosmolarity, which even leads to periodic coma",
    "Mike has kuru."
]


def genWordVec(text):
    x = np.zeros(len(wordSet))
    text = cleanText(text)
    for word in text.split():
        if word in word2idx:
            idx = word2idx[word]
            x[idx] = 1
    return x

testX = np.asarray([genWordVec(desc) for desc in testDescs])
predY = clf.predict(testX)

tgtdf[ICDCODE].iloc[predY]

1514      E0801
1525     E08319
1677    E093211
1527    E083211
3395       G723
68        A8181
Name: Diagnosis\nCode, dtype: object